In [24]:
import chardet
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

Vérifie l'encodage

In [25]:
# with open("data/houses_madrid_eda_features.csv", "rb") as fichier_brut:
#     encodage = chardet.detect(fichier_brut.read(1000))
# print(encodage)

Importe le .csv

In [26]:
# df = pd.read_csv("data/houses_madrid_eda_features.csv", encoding= 'ascii')
# df.head(1)

Exporte et réimporte en .feather (plus rapide que .csv)

In [27]:
# df.to_feather(r'data/houses_madrid_eda_features.feather')
# df = pd.read_feather("data/houses_madrid_eda_features.feather")
df = pd.read_feather("data_model/houses.feather")
df.head(1)

,log_buy_price,sq_mt_built,n_rooms,n_bathrooms,neighborhood,has_lift,has_parking,has_pool,has_garden,has_storage_room,is_floor_under
0,11.350418,64.0,2,1.0,135,0.0,0,0,0,0,0.0


Séparation variables explicatives et variable dépendante
Split train et Test

In [28]:
X = df.drop(columns=['log_buy_price'])
y = df['log_buy_price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Variables vides

In [29]:
empty_variables = X_train.columns[X_train.isna().all()].tolist()
empty_variables


[]

Taux de remplissage des variables

In [30]:
fill_rate = X_train.count() / len(X_train) * 100
fill_rate

sq_mt_built         100.0
n_rooms             100.0
n_bathrooms         100.0
neighborhood        100.0
has_lift            100.0
has_parking         100.0
has_pool            100.0
has_garden          100.0
has_storage_room    100.0
is_floor_under      100.0
dtype: float64

In [31]:
# Imputation par régression itérative (IterativeImputer)
imputer = IterativeImputer(random_state=42, max_iter=10)

# ===== TRAIN =====
X_train_imputed = pd.DataFrame(
    imputer.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)
print("✅ Imputation par régression de X_train complétée")
print(f"Valeurs manquantes restantes: {X_train_imputed.isna().sum().sum()}")

# ===== TEST =====
X_test_imputed = pd.DataFrame(
    # ✅ PAS de fit ici, seulement transform
    imputer.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

print("✅ Imputation par régression de X_test complétée")
print(f"Valeurs manquantes restantes: {X_test_imputed.isna().sum().sum()}")

✅ Imputation par régression de X_train complétée
Valeurs manquantes restantes: 0
✅ Imputation par régression de X_test complétée
Valeurs manquantes restantes: 0


Shape avant split

In [32]:
df.shape


(21454, 11)

Shape apres split (train_set)

In [33]:
X_train_imputed.shape

(17163, 10)

Shape apres split (test_set)

In [34]:
X_test_imputed.shape

(4291, 10)

EXPORTS

In [35]:
# Train
X_train_imputed.reset_index(drop=True).to_feather("data_model/X_train.feather")
y_train.reset_index(drop=True).to_frame("log_buy_price").to_feather("data_model/y_train.feather")

# Test
X_test_imputed.reset_index(drop=True).to_feather("data_model/X_test.feather")
y_test.reset_index(drop=True).to_frame("log_buy_price").to_feather("data_model/y_test.feather")